## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [25]:
import random
import pandas as pd
import string
import time
from collections import defaultdict
import numpy as np
from numba import njit


## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]
B = [a + 100 for a in A]
srzB = sum(B) / N 
print(srzB)

600.375956


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [3]:
# создаем таблицу
n_rows = 2000000
df = pd.DataFrame({'Цифры1': [random.random() for _ in range(n_rows)],
                   'Цифры2': [random.random() for _ in range(n_rows)],
                   'Цифры3': [random.random() for _ in range(n_rows)],
                   'Цифры4': [random.random() for _ in range(n_rows)]})

# добавляем столбец key со случайными английскими буквами
def random_letter():
    return random.choice(string.ascii_lowercase)

df['Буквы'] = [''.join([random_letter() for _ in range(5)]) for _ in range(n_rows)]

# выбираем подмножество строк с первыми 5 буквами в столбце key
subset = df[df['Буквы'].str[:5] == 'abcde']

print(df)
print(subset)

           Цифры1    Цифры2    Цифры3    Цифры4  Буквы
0        0.934167  0.752578  0.901822  0.809557  trgco
1        0.813519  0.873932  0.147907  0.496128  wztst
2        0.252431  0.026650  0.642659  0.611808  wqwbx
3        0.124076  0.645989  0.500388  0.894639  iumub
4        0.885419  0.808487  0.951059  0.583032  miqqp
...           ...       ...       ...       ...    ...
1999995  0.865205  0.356799  0.013916  0.004484  rydmx
1999996  0.829167  0.241240  0.506321  0.502832  aaryw
1999997  0.379155  0.173619  0.483502  0.085626  rgyye
1999998  0.864124  0.719143  0.654138  0.820976  ryfwz
1999999  0.741591  0.279985  0.213736  0.369723  bqbqh

[2000000 rows x 5 columns]
Empty DataFrame
Columns: [Цифры1, Цифры2, Цифры3, Цифры4, Буквы]
Index: []


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
# загрузка данных
recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

# приводим столбцы к нужному типу
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

# функция подсчета среднего значения с использованием метода DataFrame.iterrows исходной таблицы
def mean_rating_iterrows(reviews):
    total = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
        if count > 0:
            return total/count
        else:
            return 0
        
# функция подсчета среднего значения с использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год
def mean_rating_iterrows_2010(reviews):
    total = 0
    count = 0
    for index, row in reviews[reviews['date'].dt.year == 2010].iterrows():
        total += row['rating']
        count += 1
        if count > 0:
            return total/count
        else:
            return 0
        
# функция подсчета среднего значения с использованием метода Series.mean
def mean_rating_mean(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

# проверка корректности работы функций и измерение времени выполнения
start_time = time.time()
result_1 = mean_rating_iterrows(reviews)
print(f"Результат функции mean_rating_iterrows: {result_1}, время выполнения: {time.time() - start_time} сек")

start_time = time.time()
result_2 = mean_rating_iterrows_2010(reviews)
print(f"Результат функции mean_rating_iterrows_2010: {result_2}, время выполнения: {time.time() - start_time} сек")

start_time = time.time()
result_3 = mean_rating_mean(reviews)
print(f"Результат функции mean_rating_mean: {result_3}, время выполнения: {time.time() - start_time} сек")



Результат функции mean_rating_iterrows: 0, время выполнения: 0.23196816444396973 сек
Результат функции mean_rating_iterrows_2010: 5.0, время выполнения: 0.03599834442138672 сек
Результат функции mean_rating_mean: 4.4544402182900615, время выполнения: 0.020002126693725586 сек


AssertionError: 

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [13]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
def get_word_reviews_count(df):
    word_reviews = defaultdict(list)
    for _, row in df.dropna(subset=['review']).iterrows():
        words = row['review'].split(' ')
        for word in words:
            word_reviews[word].append(row['recipe_id'])
            
word_reviews_count = {}
for word in word_reviews:
    word_reviews_count[word] = len(word_reviews[word])
return word_reviews_count

Неоптимальности:
1. Два цикла по строкам DataFrame, которые идентичны друг другу. Это явная дубликация кода.
2. Использование словаря для хранения списков recipe_id, соответствующих каждому слову. Если слово встречается много раз в отзывах, то приходится много раз обращаться к списку, что ведет к дополнительным затратам времени.
3. Использование len(word_reviews[word]) для подсчета количества отзывов, содержащих определенное слово. Эта операция занимает O(n), где n - длина списка recipe_id. Если список очень длинный, то это может сильно замедлить работу программы.

Оптимальности:

1. Объединение двух циклов в один, чтобы избежать дубликации кода.
2. Использование defaultdict(list) вместо обычного словаря для хранения recipe_id, соответствующих каждому слову. defaultdict(list) создает новый список автоматически, если ключ еще не существует в словаре. Таким образом, мы избавляемся от необходимости делать проверку наличия ключа.
3. Использование len(word_reviews[word]) для подсчета количества recipe_id заменяется на len(set(word_reviews[word])). Таким образом, мы сначала преобразуем список recipe_id в множество (set), что убирает дубликаты и сокращает длину списка. Затем мы считаем количество элементов в множестве, что занимает O(1) и не зависит от длины списка.

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [29]:
# 1 столбец рейтинг, 2 столбец кол-во лайков
reviews = np.array([[4, 10], [3, 5], [5, 15], [2, 0], [4, 8]])

# средний рейтинг по всем отзывам
srzR = np.mean(reviews[:, 0])
print(srzR)

# удаляем отзывы с нулевым рейтингом
reviews = reviews[reviews[:, 0] > 0, :]

# MAPE без использования векторизованных операций
def mape_py(reviews):
    mape = 0
    for review in reviews:
        rating = review[0]
        likes = review[1]
        mape += abs(rating - srzR) / rating
    mape /= len(reviews)
    return mape
    
print(mape_py(reviews))

# MAPE с использованием numba
@njit
def mape_numba(reviews):
    mape = 0
    for review in reviews:
        rating = review[0]
        likes = review[1]
        mape += abs(rating - srzR) / rating
    mape /= len(reviews)
    return mape

print(mape_numba(reviews))

# MAPE с использованием numpy
def mape_np(reviews):
    ratings = reviews[:, 0]
    likes = reviews[:, 1]
    mape = np.mean(np.abs(ratings - srzR) / ratings)
    return mape

print(mape_np(reviews))

# MAPE с использованием numpy и numba
@njit
def mape_np_numba(reviews):
    ratings = reviews[:, 0]
    likes = reviews[:, 1]
    mape = np.mean(np.abs(ratings - srzR) / ratings)
    return mape

print(mape_np_numba(reviews))

3.6
0.296
0.296
0.296
0.296
